<a href="https://colab.research.google.com/github/Kalazclint/HdayBuzz-ContentClassifier/blob/main/Holiday_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
csv_file = '/content/drive/My Drive/dataset/train.csv'

In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim


In [ ]:
df = pd.read_csv(csv_file)
df.head()


,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.class_to_idx = {class_name: i for i, class_name in enumerate(self.df['Class'].unique())}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.df.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')  # Convert image to RGB
        label = self.class_to_idx[self.df.iloc[idx, 1]]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
# Define the transformations: resizing the images to 128x128 and converting them to tensors
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Create datasets
dataset = CustomImageDataset(csv_file= '/content/drive/My Drive/dataset/train.csv', img_dir='/content/drive/My Drive/dataset/train/', transform=transform)

# Split data into training, validation, and test sets and create dataloaders
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=6):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)  # You might need to adjust this value
        self.fc = nn.Linear(1024, num_classes)  # Adjusted input size to 1024

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(conv3x3(self.in_channels, out_channels, stride=stride),
                                       nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [ ]:
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ResNet(ResidualBlock, [2, 2, 2], num_classes=6).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
num_epochs = 20  # Adjust as necessary

for epoch in range(num_epochs):
    model.train()

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/20], Step [10/142], Loss: 1.3724
Epoch [1/20], Step [20/142], Loss: 1.3393
Epoch [1/20], Step [30/142], Loss: 1.4076
Epoch [1/20], Step [40/142], Loss: 1.2712
Epoch [1/20], Step [50/142], Loss: 1.4227
Epoch [1/20], Step [60/142], Loss: 1.2671
Epoch [1/20], Step [70/142], Loss: 0.8903
Epoch [1/20], Step [80/142], Loss: 1.2408
Epoch [1/20], Step [90/142], Loss: 1.5224
Epoch [1/20], Step [100/142], Loss: 0.9878
Epoch [1/20], Step [110/142], Loss: 1.0773
Epoch [1/20], Step [120/142], Loss: 1.1541
Epoch [1/20], Step [130/142], Loss: 1.0569
Epoch [1/20], Step [140/142], Loss: 1.0634
Epoch [2/20], Step [10/142], Loss: 1.2244
Epoch [2/20], Step [20/142], Loss: 1.1267
Epoch [2/20], Step [30/142], Loss: 1.1435
Epoch [2/20], Step [40/142], Loss: 1.0825
Epoch [2/20], Step [50/142], Loss: 0.8395
Epoch [2/20], Step [60/142], Loss: 1.2724
Epoch [2/20], Step [70/142], Loss: 1.2566
Epoch [2/20], Step [80/142], Loss: 1.1618
Epoch [2/20], Step [90/142], Loss: 0.9679
Epoch [2/20], Step [100/142],

In [ ]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)

        # Total number of labels
        total += labels.size(0)

        # Total correct predictions
        correct += (predicted == labels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')


Accuracy: 71.13%
